<a href="https://colab.research.google.com/github/ChintPatel/CMPE258-HW8/blob/main/HW9_Transformers_and_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet tensorflow tensorflow-text tensorflow-hub transformers datasets keras-nlp

1. Inference with a Pretrained Classifier

In [2]:
from transformers import pipeline

# — Text Generation (GPT-2) —
gen = pipeline("text-generation", model="gpt2")
print("=== Text Generation ===")
print(gen("In a world where AI rules,", max_length=50, do_sample=True)[0]["generated_text"])

# — Sentiment Analysis (DistilBERT) —
clf = pipeline("sentiment-analysis")
print("\n=== Sentiment Analysis ===")
for txt in ["I love this movie!", "This was the worst book I’ve read."]:
    print(f"{txt} ->", clf(txt)[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu



=== Sentiment Analysis ===
I love this movie! -> {'label': 'POSITIVE', 'score': 0.9998775720596313}
This was the worst book I’ve read. -> {'label': 'NEGATIVE', 'score': 0.9997784495353699}


2. Fine-tuning a Pretrained Backbone (DistilBERT → IMDb)

In [4]:
# Cell 2: Load IMDb via TensorFlow Datasets instead of 🤗datasets
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizerFast

# 1. Download & split
(raw_train, raw_test), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# 2. Prepare tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
max_length = 128

print(f"✅ Loaded IMDb: {ds_info.splits['train'].num_examples} train, "
      f"{ds_info.splits['test'].num_examples} test")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.IAHUI2_1.0.0/imdb_reviews-train.tfrecor…

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

✅ Loaded IMDb: 25000 train, 25000 test


Build & Train Your Own Transformer from Scratch

In [22]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

# 1. Load & split IMDb from TFDS
all_train = tfds.load('imdb_reviews', split='train', as_supervised=True)
all_train = all_train.shuffle(10_000, seed=42)
train_ds  = all_train.take(5000)
val_ds    = all_train.skip(5000).take(2000)

# 2. Text Vectorization layer
max_features  = 20_000
sequence_len  = 200
vectorize = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_len
)
# Adapt on the text-only stream
vectorize.adapt(train_ds.map(lambda text, label: text))

# 3. Define a single Transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.attn       = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn        = keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim),
        ])
        self.ln1        = layers.LayerNormalization()
        self.ln2        = layers.LayerNormalization()
        self.drop1      = layers.Dropout(rate)
        self.drop2      = layers.Dropout(rate)

    # Modified call method to accept *args and **kwargs
    def call(self, inputs, training=None, *args, **kwargs):
        attn_out = self.attn(inputs, inputs)
        # Pass training to Dropout layers explicitly
        attn_out = self.drop1(attn_out, training=training)
        out1     = self.ln1(inputs + attn_out)
        ffn_out  = self.ffn(out1)
        # Pass training to Dropout layers explicitly
        ffn_out  = self.drop2(ffn_out, training=training)
        return self.ln2(out1 + ffn_out)

# 4. Build the model
embed_dim = 64
num_heads = 2
ff_dim    = 64

inputs = layers.Input(shape=(sequence_len,), dtype='int64')
x = layers.Embedding(max_features, embed_dim)(inputs)
# The functional API should pass 'training' implicitly now
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x) # This Dropout layer will receive 'training' implicitly
x = layers.Dense(20, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

scratch_model = keras.Model(inputs, outputs)
scratch_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 5. Prepare tf.data pipelines
def prepare(ds):
    return (
        ds
        .map(lambda text, label: (vectorize(text), label),
             num_parallel_calls=tf.data.AUTOTUNE)
        .batch(32)
        .prefetch(tf.data.AUTOTUNE)
    )

tf_train = prepare(train_ds)
tf_val   = prepare(val_ds)

# 6. Train!
scratch_model.fit(
    tf_train,
    validation_data=tf_val,
    epochs=3
)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 236ms/step - accuracy: 0.5838 - loss: 0.6665 - val_accuracy: 0.8570 - val_loss: 0.3634
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 240ms/step - accuracy: 0.8317 - loss: 0.3717 - val_accuracy: 0.8835 - val_loss: 0.3016
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 43s 256ms/step - accuracy: 0.9028 - loss: 0.2560 - val_accuracy: 0.8990 - val_loss: 0.2407
